In [1]:
import geopandas as gpd
import pandas as pd
import numpy as np

import rasterstats

import stackstac
import pystac_client

In [2]:
path_to_aoi = r"\\storage2\production\Finland\RD_projects\KUOPUS_2022\Species_classification\geopackage\stands_AOI.gpkg"
aoi_layer_name = "stands_AOI"

aoi_data = gpd.read_file(path_to_aoi, layer=aoi_layer_name)
aoi_data_wgs = aoi_data.to_crs(crs="EPSG:4326")

bbox = list(aoi_data.bounds.loc[0])
bbox_wgs = list(aoi_data_wgs.bounds.loc[0])

In [3]:
print(bbox)
print(bbox_wgs)

[596000.0000000009, 6912000.000000004, 614000.0000000009, 6936000.0]
[28.85293582102271, 62.32208325626651, 29.215953867109075, 62.542453897106675]


In [4]:
URL = "https://paituli.csc.fi/geoserver/ogc/stac/v1"
catalog = pystac_client.Client.open(URL)
catalog

<Client id=GeoserverSTACLandingPage>

In [5]:
collections = [collection.id for collection in catalog.get_all_collections()]
collections.sort()

_ = [print(collection) for collection in collections]

2m_digital_terrain_model_products_at_fmi
building_at_geocubes
canopy_cover_at_geocubes
canopy_height_model_at_fmi
corine_land_cover_at_geocubes
country_at_geocubes
country_land_at_geocubes
daily_wind_damage_risk_at_fmi
digital_surface_model_at_geocubes
digital_terrain_model_10m_at_geocubes
digital_terrain_model_2m_aspect_at_geocubes
digital_terrain_model_2m_at_geocubes
digital_terrain_model_2m_slope_at_geocubes
fmi_daily_global_radiation_10km_at_paituli
fmi_daily_max_temperature_10km_at_paituli
fmi_daily_max_temperature_predictions_10km_at_paituli
fmi_daily_mean_temperature_10km_at_paituli
fmi_daily_mean_temperature_predictions_10km_at_paituli
fmi_daily_min_temperature_10km_at_paituli
fmi_daily_min_temperature_predictions_10km_at_paituli
fmi_daily_precipitation_10km_at_paituli
fmi_daily_precipitation_predictions_10km_at_paituli
fmi_daily_relative_humidity_10km_at_paituli
fmi_daily_sea_level_pressure_10km_at_paituli
fmi_daily_snow_depth_10km_at_paituli
fmi_monthly_avg_temperature_1km_at

In [6]:
def get_data(*, catalog, params, assets, bbox, epsg, resolution, time_period):

    search = catalog.search(**params)
    item_collection = search.item_collection()
    
    num_found_items = len(item_collection.items)
    print(f'Found items: {num_found_items}')

    gdf = None
    if num_found_items > 0:
        # ItemCollection as GeoJson 
        stac_json = search.item_collection_as_dict()
        
        # Add Item ID to properties to have access to it in GeoPandas
        for a in stac_json['features']:
            a['properties']['title']=a['id']
        
        # GeoJson as GeoPandas dataframe
        gdf = gpd.GeoDataFrame.from_features(stac_json, f"epsg:{epsg}")

    stack = stackstac.stack(
        items=item_collection,
        assets=assets,
        bounds=bbox,
        epsg=epsg,
        resolution=resolution
    ).squeeze()

    resampled = stack.resample(time=time_period).median("time", skipna=True, keep_attrs=True)

    return gdf, stack, resampled

In [7]:
s2_11_d_params = {
    "bbox": bbox_wgs,
    "collections": "sentinel_2_11_days_mosaics_at_fmi",
    "datetime": "2023-05-01/2023-10-31"
}

s2_m_index_params = {
    "bbox": bbox_wgs,
    "collections": "sentinel_2_monthly_index_mosaics_at_fmi",
    "datetime": "2023-05-01/2023-10-31"
}

s2_11_d_gdf, s2_11_d_stack, s2_11_d_monthly = get_data(
    catalog=catalog,
    params=s2_11_d_params,
    assets=['b02', 'b03', 'b04', 'b05', 'b06', 'b07', 'b08', 'b8a', 'b11', 'b12'],
    bbox=bbox,
    epsg=3067,
    resolution=10,
    time_period="MS"
)

s2_m_index_gdf, s2_m_index_stack, s2_m_index_monthly = get_data(
    catalog=catalog,
    params=s2_m_index_params,
    assets=["meta", "ndvi", "ndti", "ndsi", "ndmi", "ndbi"],
    bbox=bbox,
    epsg=3067,
    resolution=10,
    time_period="MS"
)

Found items: 12
Found items: 12


In [8]:
display(s2_11_d_gdf)
display(s2_m_index_gdf)

,geometry,start_datetime,end_datetime,datetime,gsd,proj:epsg,proj:transform,title
0,"POLYGON ((19.117 58.68, 31.59 58.84, 33.993 70...",2023-10-21T00:00:00.000+00:00,2023-10-31T23:59:59.000+00:00,2023-10-21T00:00:00.000+00:00,20.0,32635,"[20, 0, 43547, 0, -20, 7795461, 0, 0, 1]",Sentinel-2_global_mosaic_dekadi_2023-10-21_202...
1,"POLYGON ((19.117 58.68, 31.59 58.84, 33.993 70...",2023-10-11T00:00:00.000+00:00,2023-10-20T23:59:59.000+00:00,2023-10-11T00:00:00.000+00:00,20.0,32635,"[20, 0, 43547, 0, -20, 7795461, 0, 0, 1]",Sentinel-2_global_mosaic_dekadi_2023-10-11_202...
2,"POLYGON ((19.117 58.68, 31.59 58.84, 33.993 70...",2023-09-21T00:00:00.000+00:00,2023-09-30T23:59:59.000+00:00,2023-09-21T00:00:00.000+00:00,20.0,32635,"[20, 0, 43547, 0, -20, 7795461, 0, 0, 1]",Sentinel-2_global_mosaic_dekadi_2023-09-21_202...
3,"POLYGON ((19.117 58.68, 31.59 58.84, 33.993 70...",2023-09-11T00:00:00.000+00:00,2023-09-20T23:59:59.000+00:00,2023-09-11T00:00:00.000+00:00,20.0,32635,"[20, 0, 43547, 0, -20, 7795461, 0, 0, 1]",Sentinel-2_global_mosaic_dekadi_2023-09-11_202...
4,"POLYGON ((19.117 58.68, 31.59 58.84, 33.993 70...",2023-09-01T00:00:00.000+00:00,2023-09-10T23:59:59.000+00:00,2023-09-01T00:00:00.000+00:00,20.0,32635,"[20, 0, 43547, 0, -20, 7795461, 0, 0, 1]",Sentinel-2_global_mosaic_dekadi_2023-09-01_202...
5,"POLYGON ((19.117 58.68, 31.59 58.84, 33.993 70...",2023-08-21T00:00:00.000+00:00,2023-08-31T23:59:59.000+00:00,2023-08-21T00:00:00.000+00:00,20.0,32635,"[20, 0, 43547, 0, -20, 7795461, 0, 0, 1]",Sentinel-2_global_mosaic_dekadi_2023-08-21_202...
6,"POLYGON ((19.117 58.68, 31.59 58.84, 33.993 70...",2023-08-11T00:00:00.000+00:00,2023-08-20T23:59:59.000+00:00,2023-08-11T00:00:00.000+00:00,20.0,32635,"[20, 0, 43547, 0, -20, 7795461, 0, 0, 1]",Sentinel-2_global_mosaic_dekadi_2023-08-11_202...
7,"POLYGON ((19.117 58.68, 31.59 58.84, 33.993 70...",2023-08-01T00:00:00.000+00:00,2023-08-10T23:59:59.000+00:00,2023-08-01T00:00:00.000+00:00,20.0,32635,"[20, 0, 43547, 0, -20, 7795461, 0, 0, 1]",Sentinel-2_global_mosaic_dekadi_2023-08-01_202...
8,"POLYGON ((19.117 58.68, 31.59 58.84, 33.993 70...",2023-07-21T00:00:00.000+00:00,2023-07-31T23:59:59.000+00:00,2023-07-21T00:00:00.000+00:00,20.0,32635,"[20, 0, 43547, 0, -20, 7795461, 0, 0, 1]",Sentinel-2_global_mosaic_dekadi_2023-07-21_202...
9,"POLYGON ((19.117 58.68, 31.59 58.84, 33.993 70...",2023-07-11T00:00:00.000+00:00,2023-07-20T23:59:59.000+00:00,2023-07-11T00:00:00.000+00:00,20.0,32635,"[20, 0, 43547, 0, -20, 7795461, 0, 0, 1]",Sentinel-2_global_mosaic_dekadi_2023-07-11_202...


,geometry,start_datetime,end_datetime,datetime,gsd,proj:epsg,proj:transform,title
0,"POLYGON ((19.087 59.298, 33.028 59.399, 36.022...",2023-10-01T00:00:00.000+00:00,2023-10-31T23:59:59.000+00:00,2023-10-01T00:00:00.000+00:00,10.0,3067,"[10, 0, 50000, 0, -10, 7800000, 0, 0, 1]",Sentinel-2_indeksimosaiikki_2023-10-01_2023-10-31
1,"POLYGON ((19.087 59.298, 33.028 59.399, 36.022...",2023-09-15T00:00:00.000+00:00,2023-10-15T23:59:59.000+00:00,2023-09-15T00:00:00.000+00:00,10.0,3067,"[10, 0, 50000, 0, -10, 7800000, 0, 0, 1]",Sentinel-2_indeksimosaiikki_2023-09-15_2023-10-15
2,"POLYGON ((19.087 59.298, 33.028 59.399, 36.022...",2023-09-01T00:00:00.000+00:00,2023-09-30T23:59:59.000+00:00,2023-09-01T00:00:00.000+00:00,10.0,3067,"[10, 0, 50000, 0, -10, 7800000, 0, 0, 1]",Sentinel-2_indeksimosaiikki_2023-09-01_2023-09-30
3,"POLYGON ((19.087 59.298, 33.028 59.399, 36.022...",2023-08-15T00:00:00.000+00:00,2023-09-15T23:59:59.000+00:00,2023-08-15T00:00:00.000+00:00,10.0,3067,"[10, 0, 50000, 0, -10, 7800000, 0, 0, 1]",Sentinel-2_indeksimosaiikki_2023-08-15_2023-09-15
4,"POLYGON ((19.087 59.298, 33.028 59.399, 36.022...",2023-08-01T00:00:00.000+00:00,2023-08-31T23:59:59.000+00:00,2023-08-01T00:00:00.000+00:00,10.0,3067,"[10, 0, 50000, 0, -10, 7800000, 0, 0, 1]",Sentinel-2_indeksimosaiikki_2023-08-01_2023-08-31
5,"POLYGON ((19.087 59.298, 33.028 59.399, 36.022...",2023-07-15T00:00:00.000+00:00,2023-08-15T23:59:59.000+00:00,2023-07-15T00:00:00.000+00:00,10.0,3067,"[10, 0, 50000, 0, -10, 7800000, 0, 0, 1]",Sentinel-2_indeksimosaiikki_2023-07-15_2023-08-15
6,"POLYGON ((19.087 59.298, 33.028 59.399, 36.022...",2023-07-01T00:00:00.000+00:00,2023-07-31T23:59:59.000+00:00,2023-07-01T00:00:00.000+00:00,10.0,3067,"[10, 0, 50000, 0, -10, 7800000, 0, 0, 1]",Sentinel-2_indeksimosaiikki_2023-07-01_2023-07-31
7,"POLYGON ((19.087 59.298, 33.028 59.399, 36.022...",2023-06-15T00:00:00.000+00:00,2023-07-15T23:59:59.000+00:00,2023-06-15T00:00:00.000+00:00,10.0,3067,"[10, 0, 50000, 0, -10, 7800000, 0, 0, 1]",Sentinel-2_indeksimosaiikki_2023-06-15_2023-07-15
8,"POLYGON ((19.087 59.298, 33.028 59.399, 36.022...",2023-06-01T00:00:00.000+00:00,2023-06-30T23:59:59.000+00:00,2023-06-01T00:00:00.000+00:00,10.0,3067,"[10, 0, 50000, 0, -10, 7800000, 0, 0, 1]",Sentinel-2_indeksimosaiikki_2023-06-01_2023-06-30
9,"POLYGON ((19.087 59.298, 33.028 59.399, 36.022...",2023-05-15T00:00:00.000+00:00,2023-06-15T23:59:59.000+00:00,2023-05-15T00:00:00.000+00:00,10.0,3067,"[10, 0, 50000, 0, -10, 7800000, 0, 0, 1]",Sentinel-2_indeksimosaiikki_2023-05-15_2023-06-15


In [9]:
s2_11_d_monthly

<xarray.DataArray 'stackstac-f33ddb9c2a07d53a9aab6622e3033ee5' (time: 5,
                                                                band: 10,
                                                                y: 2400, x: 1801)> Size: 2GB
dask.array<stack, shape=(5, 10, 2400, 1801), dtype=float64, chunksize=(1, 1, 1024, 1024), chunktype=numpy.ndarray>
Coordinates:
  * band            (band) <U3 120B 'b02' 'b03' 'b04' ... 'b8a' 'b11' 'b12'
  * x               (x) float64 14kB 5.96e+05 5.96e+05 ... 6.14e+05 6.14e+05
  * y               (y) float64 19kB 6.936e+06 6.936e+06 ... 6.912e+06 6.912e+06
    proj:transform  object 8B {0, 1, 7795461, -20, 20, 43547}
    proj:epsg       int64 8B 32635
    gsd             float64 8B 20.0
    title           (band) <U3 120B 'b02' 'b03' 'b04' ... 'b8a' 'b11' 'b12'
    epsg            int64 8B 3067
  * time            (time) datetime64[ns] 40B 2023-06-01 ... 2023-10-01
Attributes:
    spec:        RasterSpec(epsg=3067, bounds=(596000, 6912000, 614010, 69360...
    crs:         epsg:3067
    transform:   | 10.00, 0.00, 596000.00|\n| 0.00,-10.00, 6936000.00|\n| 0.0...
    resolution:  10

In [10]:
s2_m_index_monthly

<xarray.DataArray 'stackstac-a56342d49a6329c84fee28f9b18cfc8b' (time: 7,
                                                                band: 6,
                                                                y: 2400, x: 1801)> Size: 1GB
dask.array<stack, shape=(7, 6, 2400, 1801), dtype=float64, chunksize=(1, 1, 1024, 1024), chunktype=numpy.ndarray>
Coordinates:
  * band            (band) <U4 96B 'meta' 'ndvi' 'ndti' 'ndsi' 'ndmi' 'ndbi'
  * x               (x) float64 14kB 5.96e+05 5.96e+05 ... 6.14e+05 6.14e+05
  * y               (y) float64 19kB 6.936e+06 6.936e+06 ... 6.912e+06 6.912e+06
    proj:transform  object 8B {0, 1, 7800000, 10, 50000, -10}
    proj:epsg       int64 8B 3067
    gsd             float64 8B 10.0
    title           (band) <U4 96B 'meta' 'ndvi' 'ndti' 'ndsi' 'ndmi' 'ndbi'
    epsg            int64 8B 3067
  * time            (time) datetime64[ns] 56B 2023-04-01 ... 2023-10-01
Attributes:
    spec:        RasterSpec(epsg=3067, bounds=(596000, 6912000, 614010, 69360...
    crs:         epsg:3067
    transform:   | 10.00, 0.00, 596000.00|\n| 0.00,-10.00, 6936000.00|\n| 0.0...
    resolution:  10

In [11]:
s2_11_d_data = s2_11_d_monthly.compute()
s2_m_index_data = s2_m_index_monthly.compute()

P:\Finland\RD_projects\KUOPUS_2022\Species_classification\.venv\Lib\site-packages\dask\utils.py:77: RuntimeWarning: All-NaN slice encountered
  return func(*args, **kwargs)


In [12]:
path_to_training_data = r"\\storage2\production\Finland\RD_projects\KUOPUS_2022\Species_classification\geopackage\stands_Training_data.gpkg"
td_layer_name = "stands_Training_data"

training_data = gpd.read_file(path_to_training_data, layer=td_layer_name)
training_data.head()

,fid_1,ID,XCOORD,YCOORD,HEIGHT,SPECIES,Shape_Length,Shape_Area,INV_UNIT,PAR_UNIT,...,L_Ch3_LI5_IP2,L_Ch3_LI5_IP3,L_Ch3_LI5_IA1,L_Ch3_LI5_IA2,L_Ch3_LI5_IA3,L_Ch3_D_RF,L_Ch3_D_RL,L_Ch3_D_RI,L_Ch3_D_RO,geometry
0,2322012.0,67330,604767.870,6913829.000,20.592600,2.0,16.0,9.875,22454000,NaN,...,52234.0,56286.0,16731.787879,25634.823529,49347.000000,0.300000,0.303333,0.350000,0.046667,"MULTIPOLYGON (((604766.37 6913830.75, 604766.8..."
1,2322222.0,8147,604771.495,6913828.125,25.119101,2.0,19.5,14.375,22454001,NaN,...,49151.0,57475.0,31289.020000,30309.875000,35478.857143,0.337349,0.251004,0.319277,0.092369,"MULTIPOLYGON (((604771.37 6913830.5, 604771.87..."
2,2322523.0,8184,604776.995,6913824.375,25.944401,2.0,29.0,25.000,22454002,NaN,...,42809.0,49283.0,18965.697248,33808.674157,34692.321429,0.259259,0.328704,0.180556,0.231481,"MULTIPOLYGON (((604775.62 6913827.75, 604776.1..."
3,2322537.0,67317,604767.870,6913823.625,20.515301,2.0,17.0,10.375,22454003,NaN,...,38845.0,44482.0,14135.750000,22624.043478,29995.187500,0.348571,0.314286,0.200000,0.137143,"MULTIPOLYGON (((604769.12 6913825.75, 604769.3..."
4,2322718.0,8200,604771.870,6913822.875,25.156799,2.0,26.5,21.875,22454004,NaN,...,42721.0,49283.0,15544.655172,23551.405797,37208.435897,0.320863,0.292086,0.323741,0.063309,"MULTIPOLYGON (((604772.62 6913826.25, 604772.8..."


In [14]:
def calculate_statistics(vector_df, raster_data):
    stats_list = []

    # Loop through time dimension
    for t in raster_data:
        date_str = np.datetime_as_string(t.time.values, unit="M")
        print("Calculating statistics for", date_str)

        # Loop through bands
        for band in t:
            band_title = str(band.title.values).upper()
            print("\tCalculating statistics for", band_title)
            
            column_name = f"S2_{date_str}_{band_title}"
        
            transform = band.transform
            band_data = band.data

            stats = rasterstats.zonal_stats(vector_df.geometry,
                                            band_data,
                                            affine=transform,
                                            stats="mean",
                                            all_touched=True)
            
            stats_list.append(pd.Series(
                # Get the calculated mean values out of the dictionaries
                data=[item["mean"] for item in stats],
                index=training_data.index,
                name=f"{column_name}_MEAN"
            ))
        
        print("")

    # Combine all the series into a single DataFrame
    stats_df = pd.concat(stats_list, axis=1)
    return stats_df

In [15]:
s2_11_d_stats = calculate_statistics(training_data, s2_11_d_data)

Calculating statistics for 2023-06
	Calculating statistics for B02


P:\Finland\RD_projects\KUOPUS_2022\Species_classification\.venv\Lib\site-packages\rasterstats\io.py:335: NodataWarning: Setting nodata to -999; specify nodata explicitly
  warnings.warn(


	Calculating statistics for B03
	Calculating statistics for B04
	Calculating statistics for B05
	Calculating statistics for B06
	Calculating statistics for B07
	Calculating statistics for B08
	Calculating statistics for B8A
	Calculating statistics for B11
	Calculating statistics for B12

Calculating statistics for 2023-07
	Calculating statistics for B02
	Calculating statistics for B03
	Calculating statistics for B04
	Calculating statistics for B05
	Calculating statistics for B06
	Calculating statistics for B07
	Calculating statistics for B08
	Calculating statistics for B8A
	Calculating statistics for B11
	Calculating statistics for B12

Calculating statistics for 2023-08
	Calculating statistics for B02
	Calculating statistics for B03
	Calculating statistics for B04
	Calculating statistics for B05
	Calculating statistics for B06
	Calculating statistics for B07
	Calculating statistics for B08
	Calculating statistics for B8A
	Calculating statistics for B11
	Calculating statistics for B12


In [28]:
s2_11_d_stats

,S2_2023-06_B02_MEAN,S2_2023-06_B03_MEAN,S2_2023-06_B04_MEAN,S2_2023-06_B05_MEAN,S2_2023-06_B06_MEAN,S2_2023-06_B07_MEAN,S2_2023-06_B08_MEAN,S2_2023-06_B8A_MEAN,S2_2023-06_B11_MEAN,S2_2023-06_B12_MEAN,...,S2_2023-10_B02_MEAN,S2_2023-10_B03_MEAN,S2_2023-10_B04_MEAN,S2_2023-10_B05_MEAN,S2_2023-10_B06_MEAN,S2_2023-10_B07_MEAN,S2_2023-10_B08_MEAN,S2_2023-10_B8A_MEAN,S2_2023-10_B11_MEAN,S2_2023-10_B12_MEAN
0,1264.500000,1395.5,1222.500000,1577.000000,2809.000000,3172.000000,3353.000000,None,1853.000000,None,...,1351.750000,1390.500000,1380.000000,1549.500000,2035.750000,2179.250000,2458.50,2338.250000,1761.250000,1582.750000
1,1259.333333,1383.0,1214.000000,1562.666667,2769.333333,3122.666667,3277.000000,None,1840.000000,None,...,1361.500000,1395.500000,1388.333333,1556.833333,2056.833333,2202.166667,2476.00,2365.333333,1769.000000,1590.166667
2,1249.000000,1358.0,1197.000000,1534.000000,2690.000000,3024.000000,3125.000000,None,1814.000000,None,...,1381.000000,1405.500000,1405.000000,1571.500000,2099.000000,2248.000000,2511.00,2419.500000,1784.500000,1605.000000
3,1280.000000,1433.0,1248.000000,1620.000000,2928.000000,3320.000000,3581.000000,None,1892.000000,None,...,1322.500000,1375.500000,1355.000000,1527.500000,1972.500000,2110.500000,2406.00,2257.000000,1738.000000,1560.500000
4,1257.333333,1375.0,1210.666667,1570.000000,2778.333333,3138.333333,3260.333333,None,1837.666667,None,...,1352.833333,1385.333333,1367.166667,1551.500000,2049.666667,2200.000000,2418.00,2363.333333,1757.666667,1591.333333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3914,1298.000000,1519.0,1370.000000,1803.000000,2982.000000,3274.000000,3532.000000,None,2413.000000,None,...,1969.000000,1987.500000,1998.500000,2173.500000,2591.000000,2714.500000,3030.50,2868.500000,2527.500000,2431.000000
3915,1325.000000,1541.0,1439.333333,1954.333333,2949.666667,3231.000000,3335.666667,None,2717.333333,None,...,1836.333333,1891.333333,1873.000000,2075.166667,2579.333333,2732.333333,3009.00,2860.833333,2313.500000,2173.666667
3916,1294.000000,1501.0,1375.000000,1875.000000,3004.000000,3277.000000,3337.000000,None,2525.000000,None,...,2026.000000,2008.000000,2024.500000,2259.500000,2739.500000,2873.500000,3104.50,2994.500000,2637.500000,2491.500000
3917,1310.000000,1521.0,1412.000000,1935.000000,2961.000000,3252.000000,3318.000000,None,2670.000000,None,...,1983.500000,2043.500000,2042.000000,2354.000000,2866.500000,2987.000000,3202.00,3128.000000,2709.000000,2590.500000


In [16]:
s2_11_d_stats.isna().sum()

S2_2023-06_B02_MEAN       0
S2_2023-06_B03_MEAN       0
S2_2023-06_B04_MEAN       0
S2_2023-06_B05_MEAN       0
S2_2023-06_B06_MEAN       0
S2_2023-06_B07_MEAN       0
S2_2023-06_B08_MEAN       0
S2_2023-06_B8A_MEAN    3919
S2_2023-06_B11_MEAN       0
S2_2023-06_B12_MEAN    3919
S2_2023-07_B02_MEAN       0
S2_2023-07_B03_MEAN       0
S2_2023-07_B04_MEAN       0
S2_2023-07_B05_MEAN       0
S2_2023-07_B06_MEAN       0
S2_2023-07_B07_MEAN      60
S2_2023-07_B08_MEAN      60
S2_2023-07_B8A_MEAN      60
S2_2023-07_B11_MEAN       0
S2_2023-07_B12_MEAN       0
S2_2023-08_B02_MEAN       0
S2_2023-08_B03_MEAN       0
S2_2023-08_B04_MEAN       0
S2_2023-08_B05_MEAN       0
S2_2023-08_B06_MEAN       0
S2_2023-08_B07_MEAN       0
S2_2023-08_B08_MEAN       0
S2_2023-08_B8A_MEAN       0
S2_2023-08_B11_MEAN       0
S2_2023-08_B12_MEAN       0
S2_2023-09_B02_MEAN      72
S2_2023-09_B03_MEAN      72
S2_2023-09_B04_MEAN      72
S2_2023-09_B05_MEAN      72
S2_2023-09_B06_MEAN      72
S2_2023-09_B07_MEAN 

In [17]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    display(s2_11_d_stats.describe())

,S2_2023-06_B02_MEAN,S2_2023-06_B03_MEAN,S2_2023-06_B04_MEAN,S2_2023-06_B05_MEAN,S2_2023-06_B06_MEAN,S2_2023-06_B07_MEAN,S2_2023-06_B08_MEAN,S2_2023-06_B11_MEAN,S2_2023-07_B02_MEAN,S2_2023-07_B03_MEAN,S2_2023-07_B04_MEAN,S2_2023-07_B05_MEAN,S2_2023-07_B06_MEAN,S2_2023-07_B07_MEAN,S2_2023-07_B08_MEAN,S2_2023-07_B8A_MEAN,S2_2023-07_B11_MEAN,S2_2023-07_B12_MEAN,S2_2023-08_B02_MEAN,S2_2023-08_B03_MEAN,S2_2023-08_B04_MEAN,S2_2023-08_B05_MEAN,S2_2023-08_B06_MEAN,S2_2023-08_B07_MEAN,S2_2023-08_B08_MEAN,S2_2023-08_B8A_MEAN,S2_2023-08_B11_MEAN,S2_2023-08_B12_MEAN,S2_2023-09_B02_MEAN,S2_2023-09_B03_MEAN,S2_2023-09_B04_MEAN,S2_2023-09_B05_MEAN,S2_2023-09_B06_MEAN,S2_2023-09_B07_MEAN,S2_2023-09_B08_MEAN,S2_2023-09_B8A_MEAN,S2_2023-09_B11_MEAN,S2_2023-09_B12_MEAN,S2_2023-10_B02_MEAN,S2_2023-10_B03_MEAN,S2_2023-10_B04_MEAN,S2_2023-10_B05_MEAN,S2_2023-10_B06_MEAN,S2_2023-10_B07_MEAN,S2_2023-10_B08_MEAN,S2_2023-10_B8A_MEAN,S2_2023-10_B11_MEAN,S2_2023-10_B12_MEAN
count,3919.000000,3919.000000,3919.000000,3919.000000,3919.000000,3919.000000,3919.000000,3919.000000,3919.000000,3919.000000,3919.000000,3919.000000,3919.000000,3859.000000,3859.000000,3859.000000,3919.000000,3919.000000,3919.000000,3919.000000,3919.000000,3919.000000,3919.000000,3919.000000,3919.000000,3919.000000,3919.000000,3919.000000,3847.000000,3847.000000,3847.000000,3847.000000,3847.000000,3847.000000,3847.000000,3847.000000,3847.000000,3847.000000,3908.000000,3908.000000,3908.000000,3908.000000,3908.000000,3908.000000,3908.000000,3908.000000,3908.000000,3908.000000
mean,1251.119801,1375.654015,1228.368738,1583.808710,2579.654674,2867.520201,2970.854321,1901.489432,1365.109892,1465.526016,1320.583184,1644.429021,2642.815769,2955.026183,3019.105468,3104.335881,1880.360785,1472.885993,1181.408586,1272.905057,1167.009941,1444.349590,2350.243738,2652.027090,2690.392830,2803.262163,1734.685911,1334.316375,1477.232324,1552.927346,1441.824994,1701.823954,2470.034800,2707.797331,2746.096785,2856.327864,1837.587308,1534.547721,1379.404565,1403.839794,1346.352962,1544.314174,2104.827992,2268.356256,2397.748870,2402.802702,1661.837939,1466.529342
std,36.098334,55.389814,54.078540,97.873814,227.611089,266.330616,295.246674,202.357706,186.002786,178.610576,182.741023,194.187097,305.004814,421.531910,449.330900,453.688519,208.331768,178.199240,52.459287,61.172670,48.275739,94.941824,234.854393,282.176006,302.698629,305.058304,168.619799,89.614146,275.289228,265.276294,248.886105,279.180285,396.481743,441.977211,468.180539,482.679748,339.148311,332.836907,302.920697,287.532956,289.401939,309.987727,394.329155,433.962495,498.046252,474.695834,338.521784,323.541037
min,1177.000000,1275.000000,1156.333333,1410.000000,2099.000000,2308.000000,2334.000000,1541.666667,1129.250000,1187.500000,1092.000000,1314.000000,1901.000000,1584.000000,1553.000000,1603.000000,1391.000000,1177.000000,1084.500000,1156.000000,1085.000000,1280.000000,1879.000000,2080.000000,2018.500000,2185.000000,1453.500000,1194.000000,1042.000000,1078.000000,1036.000000,1212.000000,1598.000000,1733.000000,1656.000000,1807.500000,1288.500000,1117.000000,1001.000000,1008.000000,1006.000000,1084.000000,1214.000000,1197.000000,1188.000000,1190.000000,1040.000000,1005.000000
25%,1229.000000,1338.000000,1195.500000,1522.000000,2422.000000,2682.000000,2772.000000,1767.000000,1242.666667,1348.625000,1208.000000,1517.437500,2431.000000,2682.125000,2730.000000,2822.000000,1736.083333,1357.125000,1147.000000,1231.000000,1136.000000,1375.500000,2171.833333,2440.000000,2464.000000,2578.000000,1613.000000,1274.000000,1298.500000,1385.000000,1285.500000,1519.583333,2192.416667,2382.187500,2404.833333,2505.125000,1571.250000,1291.250000,1184.104167,1225.500000,1163.000000,1355.750000,1842.000000,1982.312500,2086.187500,2101.000000,1440.000000,1238.750000
50%,1247.000000,1362.000000,1213.000000,1563.500000,2532.666667,2817.000000,2916.500000,1856.500000,1296.333333,1405.500000,1248.000000,1605.500000,2620.000000,2937.000000,3015.000000,3097.0000

In [18]:
s2_m_index_stats = calculate_statistics(training_data, s2_m_index_data)

Calculating statistics for 2023-04
	Calculating statistics for META
	Calculating statistics for NDVI
	Calculating statistics for NDTI
	Calculating statistics for NDSI
	Calculating statistics for NDMI
	Calculating statistics for NDBI

Calculating statistics for 2023-05
	Calculating statistics for META
	Calculating statistics for NDVI
	Calculating statistics for NDTI
	Calculating statistics for NDSI
	Calculating statistics for NDMI
	Calculating statistics for NDBI

Calculating statistics for 2023-06
	Calculating statistics for META
	Calculating statistics for NDVI
	Calculating statistics for NDTI
	Calculating statistics for NDSI
	Calculating statistics for NDMI
	Calculating statistics for NDBI

Calculating statistics for 2023-07
	Calculating statistics for META
	Calculating statistics for NDVI
	Calculating statistics for NDTI
	Calculating statistics for NDSI
	Calculating statistics for NDMI
	Calculating statistics for NDBI

Calculating statistics for 2023-08
	Calculating statistics for M

In [29]:
s2_m_index_stats

,S2_2023-04_META_MEAN,S2_2023-04_NDVI_MEAN,S2_2023-04_NDTI_MEAN,S2_2023-04_NDSI_MEAN,S2_2023-04_NDMI_MEAN,S2_2023-04_NDBI_MEAN,S2_2023-05_META_MEAN,S2_2023-05_NDVI_MEAN,S2_2023-05_NDTI_MEAN,S2_2023-05_NDSI_MEAN,...,S2_2023-09_NDTI_MEAN,S2_2023-09_NDSI_MEAN,S2_2023-09_NDMI_MEAN,S2_2023-09_NDBI_MEAN,S2_2023-10_META_MEAN,S2_2023-10_NDVI_MEAN,S2_2023-10_NDTI_MEAN,S2_2023-10_NDSI_MEAN,S2_2023-10_NDMI_MEAN,S2_2023-10_NDBI_MEAN
0,133.0,151.500000,133.000000,95.250000,131.250000,17.500000,151.500000,163.500000,138.125000,91.375000,...,140.750000,102.5,140.250000,8.250000,293.0,141.000000,113.000000,101.000000,131.500000,28.000000
1,133.0,152.000000,133.000000,96.000000,132.000000,16.333333,151.500000,162.833333,138.000000,91.500000,...,140.333333,103.0,139.666667,9.000000,293.0,141.000000,113.000000,100.666667,131.000000,28.333333
2,133.0,153.500000,134.500000,98.000000,135.000000,12.000000,151.500000,161.750000,138.000000,93.500000,...,140.500000,105.0,140.500000,6.500000,293.0,143.500000,113.000000,101.000000,132.000000,25.500000
3,133.0,150.000000,133.000000,95.000000,129.000000,22.000000,151.500000,162.500000,138.000000,90.500000,...,140.000000,102.0,141.000000,10.000000,293.0,138.000000,113.000000,100.000000,128.000000,34.000000
4,133.0,152.333333,133.333333,97.000000,133.000000,15.000000,151.500000,162.166667,138.000000,92.833333,...,140.000000,104.0,140.666667,7.666667,293.0,141.000000,112.666667,100.666667,130.333333,29.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3914,133.0,140.000000,129.000000,69.000000,101.000000,60.000000,153.000000,157.500000,134.000000,71.000000,...,132.000000,103.5,131.000000,29.000000,NaN,138.000000,137.000000,107.000000,133.000000,28.000000
3915,133.0,144.000000,129.333333,67.666667,105.666667,51.333333,151.666667,156.333333,131.333333,69.500000,...,131.500000,107.5,134.500000,23.500000,293.0,125.333333,106.333333,82.000000,107.000000,68.333333
3916,133.0,139.500000,129.500000,71.000000,104.000000,57.000000,153.000000,158.250000,133.750000,72.750000,...,132.500000,105.0,131.500000,29.000000,NaN,137.000000,137.000000,109.000000,132.000000,32.000000
3917,133.0,144.000000,129.000000,67.500000,104.000000,52.500000,152.250000,157.000000,131.750000,69.250000,...,131.500000,106.5,132.500000,27.500000,NaN,128.500000,116.000000,93.000000,116.500000,56.000000


In [19]:
s2_m_index_stats.isna().sum()

S2_2023-04_META_MEAN       0
S2_2023-04_NDVI_MEAN       0
S2_2023-04_NDTI_MEAN       0
S2_2023-04_NDSI_MEAN       0
S2_2023-04_NDMI_MEAN       0
S2_2023-04_NDBI_MEAN       1
S2_2023-05_META_MEAN       0
S2_2023-05_NDVI_MEAN       0
S2_2023-05_NDTI_MEAN       0
S2_2023-05_NDSI_MEAN       0
S2_2023-05_NDMI_MEAN       0
S2_2023-05_NDBI_MEAN     441
S2_2023-06_META_MEAN       0
S2_2023-06_NDVI_MEAN       0
S2_2023-06_NDTI_MEAN       0
S2_2023-06_NDSI_MEAN       0
S2_2023-06_NDMI_MEAN       0
S2_2023-06_NDBI_MEAN    1555
S2_2023-07_META_MEAN       0
S2_2023-07_NDVI_MEAN       0
S2_2023-07_NDTI_MEAN       0
S2_2023-07_NDSI_MEAN       0
S2_2023-07_NDMI_MEAN       0
S2_2023-07_NDBI_MEAN    1476
S2_2023-08_META_MEAN       0
S2_2023-08_NDVI_MEAN       0
S2_2023-08_NDTI_MEAN       0
S2_2023-08_NDSI_MEAN       0
S2_2023-08_NDMI_MEAN       0
S2_2023-08_NDBI_MEAN    1816
S2_2023-09_META_MEAN     179
S2_2023-09_NDVI_MEAN     163
S2_2023-09_NDTI_MEAN     163
S2_2023-09_NDSI_MEAN     163
S2_2023-09_NDM

In [20]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    display(s2_m_index_stats.describe())

,S2_2023-04_META_MEAN,S2_2023-04_NDVI_MEAN,S2_2023-04_NDTI_MEAN,S2_2023-04_NDSI_MEAN,S2_2023-04_NDMI_MEAN,S2_2023-04_NDBI_MEAN,S2_2023-05_META_MEAN,S2_2023-05_NDVI_MEAN,S2_2023-05_NDTI_MEAN,S2_2023-05_NDSI_MEAN,S2_2023-05_NDMI_MEAN,S2_2023-05_NDBI_MEAN,S2_2023-06_META_MEAN,S2_2023-06_NDVI_MEAN,S2_2023-06_NDTI_MEAN,S2_2023-06_NDSI_MEAN,S2_2023-06_NDMI_MEAN,S2_2023-06_NDBI_MEAN,S2_2023-07_META_MEAN,S2_2023-07_NDVI_MEAN,S2_2023-07_NDTI_MEAN,S2_2023-07_NDSI_MEAN,S2_2023-07_NDMI_MEAN,S2_2023-07_NDBI_MEAN,S2_2023-08_META_MEAN,S2_2023-08_NDVI_MEAN,S2_2023-08_NDTI_MEAN,S2_2023-08_NDSI_MEAN,S2_2023-08_NDMI_MEAN,S2_2023-08_NDBI_MEAN,S2_2023-09_META_MEAN,S2_2023-09_NDVI_MEAN,S2_2023-09_NDTI_MEAN,S2_2023-09_NDSI_MEAN,S2_2023-09_NDMI_MEAN,S2_2023-09_NDBI_MEAN,S2_2023-10_META_MEAN,S2_2023-10_NDVI_MEAN,S2_2023-10_NDTI_MEAN,S2_2023-10_NDSI_MEAN,S2_2023-10_NDMI_MEAN,S2_2023-10_NDBI_MEAN
count,3919.000000,3919.000000,3919.000000,3919.000000,3919.000000,3918.000000,3919.000000,3919.000000,3919.000000,3919.000000,3919.000000,3478.000000,3919.000000,3919.000000,3919.000000,3919.000000,3919.000000,2364.000000,3919.000000,3919.000000,3919.000000,3919.000000,3919.000000,2443.000000,3919.000000,3919.000000,3919.000000,3919.000000,3919.000000,2103.000000,3740.000000,3756.000000,3756.000000,3756.000000,3756.000000,3541.000000,3896.000000,3905.000000,3905.000000,3905.000000,3905.000000,3350.000000
mean,132.543697,153.798269,135.520222,89.465170,128.884643,17.707057,151.863826,162.163264,137.418729,90.382698,136.862061,6.893509,171.802447,164.491675,138.565716,91.123841,139.806530,4.337017,204.711651,163.094965,140.853428,94.515384,140.928394,5.466230,233.188887,162.886281,142.559773,96.772572,141.875032,5.797987,261.776649,148.240049,137.281794,113.721778,143.725861,11.851325,290.290533,147.207981,134.961353,118.437196,145.765749,13.462488
std,1.331648,3.600641,2.082724,8.478218,7.061031,9.080573,2.547408,3.455146,1.851928,7.665305,5.979198,4.911279,5.263870,3.410690,1.921823,6.966536,5.213624,3.599205,5.540039,4.571936,2.418707,8.270009,5.119889,4.427283,4.546601,4.237408,2.212503,9.376171,5.684823,5.108855,8.161823,8.497580,4.697192,12.905200,6.132129,8.220430,5.317392,8.117270,7.723347,12.894862,8.930892,11.393088
min,115.000000,128.000000,124.666667,56.666667,86.666667,1.500000,138.000000,149.500000,127.333333,56.000000,97.000000,1.000000,162.500000,153.500000,128.125000,58.166667,103.500000,1.000000,189.500000,145.500000,124.750000,61.666667,106.333333,1.000000,218.000000,146.500000,132.375000,60.500000,103.250000,1.000000,243.000000,115.875000,110.000000,80.750000,122.500000,1.000000,278.000000,117.500000,96.000000,82.000000,106.750000,1.000000
25%,133.000000,152.000000,134.500000,83.500000,125.000000,12.000000,150.500000,159.750000,136.500000,85.000000,133.500000,4.000000,168.000000,162.250000,137.500000,86.500000,137.000000,2.000000,201.125000,160.500000,139.500000,89.000000,138.062500,2.750000,230.500000,160.500000,141.500000,90.000000,139.000000,2.500000,256.000000,144.500000,135.500000,104.500000,140.125000,5.375000,287.729167,143.750000,134.666667,110.000000,141.666667,6.000000
50%,133.000000,154.000000,136.000000,90.500000,129.500000,16.500000,151.500000,162.000000,137.500000,91.000000,138.000000,5.750000,172.625000,164.500000,138.666667,91.833333,140.500000,3.500000,205.500000,163.500000,141.000000,94.500000,141.250000,4.250000,234.250000,163.333333,142.750000,97.000000,142.250000,4.250000,260.000000,149.250000,138.000000,112.000000,144.000000,9.500000,293.000000,148.333333,136.500000,118.000000,147.250000,10.125000
75%,133.000000,156.000000,137.000000,96.000000,134.000000,21.500000,153.750000,164.000000,138.500000,96.250000,140.666667,8.250000,175.750000,166.500000,140.000000,96.500000,143.000000,5.000000,206.500000,166.000000,142.500000,100.000000,144.000000,6.750000,238.000000,166.000000,144.000000,103.500000,145.500000,7.500000,265.000000,154.500000,140.000000,121.000000,147.500000,17.000000,293.000000,153.000000,138.500000,126.000000,1

In [21]:
training_data_all_stats = pd.concat([training_data, s2_11_d_stats, s2_m_index_stats], axis=1)

In [22]:
training_data_all_stats

,fid_1,ID,XCOORD,YCOORD,HEIGHT,SPECIES,Shape_Length,Shape_Area,INV_UNIT,PAR_UNIT,...,S2_2023-09_NDTI_MEAN,S2_2023-09_NDSI_MEAN,S2_2023-09_NDMI_MEAN,S2_2023-09_NDBI_MEAN,S2_2023-10_META_MEAN,S2_2023-10_NDVI_MEAN,S2_2023-10_NDTI_MEAN,S2_2023-10_NDSI_MEAN,S2_2023-10_NDMI_MEAN,S2_2023-10_NDBI_MEAN
0,2322012.0,67330,604767.870,6913829.000,20.592600,2.0,16.0,9.8750,22454000,NaN,...,140.750000,102.5,140.250000,8.250000,293.0,141.000000,113.000000,101.000000,131.500000,28.000000
1,2322222.0,8147,604771.495,6913828.125,25.119101,2.0,19.5,14.3750,22454001,NaN,...,140.333333,103.0,139.666667,9.000000,293.0,141.000000,113.000000,100.666667,131.000000,28.333333
2,2322523.0,8184,604776.995,6913824.375,25.944401,2.0,29.0,25.0000,22454002,NaN,...,140.500000,105.0,140.500000,6.500000,293.0,143.500000,113.000000,101.000000,132.000000,25.500000
3,2322537.0,67317,604767.870,6913823.625,20.515301,2.0,17.0,10.3750,22454003,NaN,...,140.000000,102.0,141.000000,10.000000,293.0,138.000000,113.000000,100.000000,128.000000,34.000000
4,2322718.0,8200,604771.870,6913822.875,25.156799,2.0,26.5,21.8750,22454004,NaN,...,140.000000,104.0,140.666667,7.666667,293.0,141.000000,112.666667,100.666667,130.333333,29.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3914,24213332.0,7286,607496.370,6935673.500,21.703899,1.0,19.0,17.0625,22457914,NaN,...,132.000000,103.5,131.000000,29.000000,NaN,138.000000,137.000000,107.000000,133.000000,28.000000
3915,24213454.0,7303,607451.370,6935670.875,22.870600,1.0,15.5,10.6250,22457915,NaN,...,131.500000,107.5,134.500000,23.500000,293.0,125.333333,106.333333,82.000000,107.000000,68.333333
3916,24213483.0,7296,607483.745,6935671.875,23.334999,2.0,18.5,12.7500,22457916,NaN,...,132.500000,105.0,131.500000,29.000000,NaN,137.000000,137.000000,109.000000,132.000000,32.000000
3917,24213693.0,7322,607466.370,6935668.375,26.629000,2.0,28.0,31.3750,22457917,NaN,...,131.500000,106.5,132.500000,27.500000,NaN,128.500000,116.000000,93.000000,116.500000,56.000000


In [ ]:
#td_out_path = r"\\storage2\production\Finland\RD_projects\KUOPUS_2022\Species_classification\All_stats\training_data_all_stats.gpkg"
#training_data_all_stats.to_file(td_out_path, layer=td_layer_name, driver="GPKG")

In [27]:
import rioxarray
s2_m_index_monthly.sel(time="2023-07-01").sel(band="ndvi").rio.to_raster(
    r"\\storage2\production\Finland\RD_projects\KUOPUS_2022\Species_classification\Sentinel\Stack\S2_2023_07_NDVI.tif"
)

In [30]:
s2_m_index_monthly.sel(time="2023-05-01").sel(band="ndsi").rio.to_raster(
    r"\\storage2\production\Finland\RD_projects\KUOPUS_2022\Species_classification\Sentinel\Stack\S2_2023_05_NDSI.tif"
)